In [440]:
import re

In [441]:
def parse_facts(path):
    parsed_facts = []
    with open(path, 'r') as file:
        for line in file:
            parsed_facts.append(re.split(r'\s+', line.strip()))
    facts = {}
    for fact in parsed_facts:
        if len(fact) == 1:
            facts[fact[0]] = True
        elif len(fact) == 3:
            facts[fact[0]] = fact[2]
    return facts
forward_facts = parse_facts('facts.txt')
backward_facts = parse_facts('facts.txt')

print("Forward Facts:" ,forward_facts)
print("Backward Facts:" ,backward_facts)

Forward Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange'}
Backward Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange'}


In [442]:
def parse_rules(path):
    parsed_rules = []
    with open(path, 'r') as rules:
        lines = rules.readlines()  
        for line in lines:
            parsed_rules.append(line.strip())

    condition = []
    conclusion = []
    for i in parsed_rules:  
        if "IF" in i and "THEN" in i:
            cond = i.split("IF")[1].split("THEN")[0].strip()
            concl = i.split("THEN")[1].strip()
            condition.append(cond)
            conclusion.append(concl)

    return parsed_rules, condition, conclusion

rules, condition, conclusion = parse_rules('rules.txt')
print("Rules:",rules)
print("Conditions:",condition)
print("Conclutions:" ,conclusion)

Rules: ['IF shape is long AND color is yellow THEN fruit is banana', 'IF shape is round AND color is red AND size is medium THEN fruit is apple', 'IF shape is round AND color is red AND size is small THEN fruit is cherry', 'IF skin_smell THEN perfumed', 'IF fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit THEN citrus_fruit', 'IF size is medium AND color is yellow AND perfumed THEN fruit is lemon', 'IF size is medium AND color is green THEN fruit is kiwi', 'IF size is big AND perfumed AND color is orange AND citrus_fruit THEN fruit is grapefruit', 'IF perfumed AND color is orange AND size is medium THEN fruit is orange', 'IF perfumed AND color is red AND size is small AND seeds = 0 THEN fruit is strawberry', 'IF diameter < 2 THEN size is small', 'IF diameter > 10 THEN size is big', 'IF diameter > 2 AND diameter < 10 THEN size is medium']
Conditions: ['shape is long AND color is yellow', 'shape is round AND color is red AND size is medium', 'shape is round AND 

In [443]:
def evaluate_expression(expr, facts):
    expr = expr.strip()

    match = re.match(r'(\w+)\s*(|<|>)\s*(\d+)', expr)
    if match:
        key, op, val = match.groups()
        if key not in facts:
            return False
        try:
            f_val = float(facts[key])
            val = float(val)
            if op == '<': return f_val < val
            if op == '>': return f_val > val
            if op == '<=': return f_val <= val
            if op == '>=': return f_val >= val
        except:
            return False

    if " is " in expr:
        key, val = expr.split(" is ")
        return facts.get(key) == val
    
    return facts.get(expr, False)

In [444]:
def match_condition(cond_str, facts):
    or_clauses = cond_str.split("OR")
    
    for or_clause in or_clauses:
        and_parts = or_clause.split("AND")
        all_true = True
        
        for part in and_parts:
            if not evaluate_expression(part.strip(), facts):
                all_true = False
                break
        
        if all_true:
            return True
    
    return False

In [445]:
def forward_chaining(facts, conditions, conclusions):
    added = True
    cycle = 0
    while added:
        cycle += 1
        added = False

        for i in range(len(conditions)):
            if match_condition(conditions[i], facts):
                if " is " in conclusions[i]:
                    key, val = conclusions[i].split(" is ")
                else:
                    key, val = conclusions[i], True

                if key not in facts:
                    facts[key] = val
                    print(f"{key} = {val} : Inferred in cycle {cycle}")
                    print("Facts:", facts)
                    added = True
                    
    return facts

results = forward_chaining(forward_facts, condition, conclusion)
print("---------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("Final Facts:", forward_facts)

perfumed = True : Inferred in cycle 1
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'perfumed': True}
size = medium : Inferred in cycle 1
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}
fruit = orange : Inferred in cycle 2
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}
citrus_fruit = True : Inferred in cycle 3
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange', 'citrus_fruit': True}
---------------------------------------------------------------------------------------------------------------------------------------------------------------
Final Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange', 'citrus_fruit': True}


In [446]:
def backward_chain_chaining(goal, facts, conditions, conclusions, visited=None, cycle_counter=None):
    if visited is None:
        visited = set()
        
    if cycle_counter is None:
        cycle_counter = {"count": 1}

    cycle_counter["count"] += 1  

    if goal in visited:
        return False  

    if goal in facts:
        return True

    visited.add(goal)

    for i in range(len(conclusions)):
        if goal in conclusions[i]:
            or_clauses = conditions[i].split("OR")
            for or_clause in or_clauses:
                and_parts = or_clause.strip().split("AND")
                all_true = True
                for part in and_parts:
                    part = part.strip()
                    if not evaluate_expression(part, facts):
                        if " is " in part:
                            value , sub_goal = part.split(" is ")
                            print(f"Sub-goal: {value} = {sub_goal} in cycle {cycle_counter['count']}")    

                        else:
                            sub_goal = part
                            print(f"Sub-goal: {sub_goal} in cycle {cycle_counter['count']}")   
                        print("-------------------------")    
                        print("Facts:", facts)     

                        if not backward_chain_chaining(sub_goal.strip(), facts, conditions, conclusions, visited, cycle_counter):
                            all_true = False
                            break
                if all_true:
                    if match_condition(conditions[i], facts):
                        if " is " in conclusions[i]:
                            key, val = conclusions[i].split(" is ")
                            val = val.strip()
                        else:
                            key, val = conclusions[i], True
                        key = key.strip()

                        if key not in facts:
                            facts[key] = val
                            print(f"{key} = {val} : Inferred in cycle {cycle_counter['count']}")
                            visited.add(key)
                            cycle_counter["count"] += 1
                            print("-------------------------")    
                            print("Facts:", facts)
        
                        return True

    return False

results = backward_chain_chaining('citrus_fruit', backward_facts, condition, conclusion, cycle_counter= None)

print("-------------------------------------------")
print("Final Facts:", backward_facts)

Sub-goal: fruit = lemon in cycle 2
-------------------------
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange'}
Sub-goal: size = medium in cycle 3
-------------------------
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange'}
size = medium : Inferred in cycle 4
-------------------------
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'size': 'medium'}
Sub-goal: color = yellow in cycle 5
-------------------------
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'size': 'medium'}
Sub-goal: fruit = orange in cycle 6
-------------------------
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'size': 'medium'}
Sub-goal: perfumed in cycle 7
-------------------------
Facts: {'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange', 'size': 'medium'}
perfumed = True : Inferred in cycle 8
-------------------------
Facts: {'seeds': '0', 'diameter'